<a href="https://colab.research.google.com/github/d-atallah/implicit_gender_bias/blob/main/Supervised_Learning_Dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Next Steps:

*  Test not removing stop words - Done, saw improvement
*  Fix Log regression issue
  * May be a class imbalance - Checked into this and found no imbalance. I tried scoring on AUC-PR and the model results were better but still not great. I believe a logistic regression may be to simple of a model to work with our data.
* Score random search on AUC-PR
  * Log regression performance was enhanced by AUC-PR score. I am now testing the best xgb model using AUC-PR. On great lakes I am running the last two random forest models.
* Use grid search rather than random search in great lakes cluster.

* Once I get the auc-pr tested XGB I will save results and try removing the stop word removal and rescore the XGB model. Once I have the best of those results I will work on using grid search with a narrowed down search range. I will do that step in great lakes for efficiency.




# Import, Download, & Variable Statements

In [ ]:
# Import & download statements
# General Statements
!git clone https://github.com/d-atallah/implicit_gender_bias.git
import pandas as pd
import string
import re
import joblib
from implicit_gender_bias import config as cf
import os
import numpy as np

# Feature selection & Model tuning
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD,PCA, NMF
from sklearn.metrics import confusion_matrix,precision_score, recall_score, f1_score, accuracy_score, roc_curve, roc_auc_score, log_loss, make_scorer, average_precision_score

# Model options
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# NLTK resources
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()

fatal: destination path 'implicit_gender_bias' already exists and is not an empty directory.


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/gibsonce/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gibsonce/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/gibsonce/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Variables
# Inputs
folder_path = '/home/gibsonce/datallah-jaymefis-gibsonce/'#cf.filepath() #'/home/gibsonce/datallah-jaymefis-gibsonce/'
csv_files = ['facebook_wiki_posts','facebook_wiki_responses','fitocracy_posts','fitocracy_responses','reddit_posts','reddit_responses','ted_responses','facebook_congress_posts','annotations','facebook_congress_responses']

annotations = pd.read_csv(folder_path+'annotations_combined.csv')
responses_combined = pd.read_csv(folder_path+'responses_combined.csv')
#posts_combined = pd.read_csv(folder_path+'posts_combined.csv')
#sources_combined = pd.read_csv(folder_path+'sources_combined_output.csv')

# Outputs
test_num = '2'

/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,4,6,7,10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Define Functions


In [ ]:
# Evaluate a model
def model_eval(model, X_test, y_test, y_pred):
    """
    Evaluates a specified model using accuracy, precision, recall, F-1 score, AUC, log-Loss, and a confusion matrix.

    Parameters:
    - model: The trained model to be evaluated.
    - X_test (list or array): Test set features.
    - y_test (list or array): True labels.
    - y_pred (list or array): Predicted labels.

    Returns:
    - metrics_df (pd.DataFrame): DataFrame containing the metrics and scores.
    - confusion_df (pd.DataFrame): DataFrame containing a confusion matrix.
    """
    # Initialize dataframes
    metrics_df = pd.DataFrame(columns=['Metric', 'Score'])
    confusion_df = pd.DataFrame(columns=['Actual Positive', 'Actual Negative', 'Predicted Positive', 'Predicted Negative'])

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    metrics_df = pd.concat([metrics_df, pd.DataFrame({'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
                                                      'Score': [accuracy, precision, recall, f1]})])

    fpr, tpr, thresholds = roc_curve(y_test, model.predict_proba(X_test)[:, 1])
    auc = roc_auc_score(y_test, y_pred)
    metrics_df = pd.concat([metrics_df, pd.DataFrame({'Metric': ['AUC'],
                                                      'Score': [auc]})])

    logloss = log_loss(y_test, model.predict_proba(X_test))
    metrics_df = pd.concat([metrics_df, pd.DataFrame({'Metric': ['Log-Loss'],
                                                      'Score': [logloss]})])

    # Reset index
    metrics_df = metrics_df.reset_index(drop=True)

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    confusion_df = pd.DataFrame(cm, columns=['Predicted Positive', 'Predicted Negative'], index=['Actual Positive', 'Actual Negative'])

    # Print dataframes
    print("Metrics:")
    print(metrics_df)

    print("\nConfusion Matrix:")
    print(confusion_df)

    return metrics_df, confusion_df

In [ ]:
stop_words = {'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 #'he',
 #'her',
 'here',
 #'hers',
 #'herself',
 #'him',
 #'himself',
 #'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 're',
 's',
 'same',
 'shan',
 "shan't",
 #'she',
 #"she's",
 'should',
 "should've",
 'shouldn',
 "shouldn't",
 'so',
 'some',
 'such',
 't',
 'than',
 'that',
 "that'll",
 'the',
 'their',
 'theirs',
 'them',
 'themselves',
 'then',
 'there',
 'these',
 'they',
 'this',
 'those',
 'through',
 'to',
 'too',
 'under',
 'until',
 'up',
 've',
 'very',
 'was',
 'wasn',
 "wasn't",
 'we',
 'were',
 'weren',
 "weren't",
 'what',
 'when',
 'where',
 'which',
 'while',
 'who',
 'whom',
 'why',
 'will',
 'with',
 'won',
 "won't",
 'wouldn',
 "wouldn't",
 'y',
 'you',
 "you'd",
 "you'll",
 "you're",
 "you've",
 'your',
 'yours',
 'yourself',
 'yourselves'}

In [ ]:
def preprocess_text(text):
    """
    Applies text preprocessing to a given text, including:
    - Removing special characters and digits
    - Converting to lowercase
    - Tokenization and removing stopwords
    - Lemmatization and stemming

    Parameters:
    - text (str): Input text to be preprocessed.

    Returns:
    - processed_text (str): Preprocessed text after applying the specified steps.
    """
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenization and removing stopwords
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization and stemming
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    tokens = [porter.stem(word) for word in tokens]

    # Rejoin tokens into a processed text
    processed_text = ' '.join(tokens)

    return processed_text

In [ ]:
def model_search(X_train, y_train, X_validation, y_validation, X_test, model_type, vectorizer, ngram, search_type, param_grid, metric_score):
  """
  Searches for the best hyperparameters for a specified model and dimensionality reduction method using GridSearchCV or RandomizedSearchCV.

  Parameters:
  - X_train (array-like): Training set features, preprocessed.
  - y_train (array-like): Training set labels.
  - X_validation (array-like): Validation set features, preprocessed.
  - y_validation (array-like): Validation set labels.
  - X_test (array-like): Test set features, preprocessed.
  - model_type (str): Type of model to test. Choose from 'log' (Logistic Regression), 'xgb' (XGBoost), or 'rf' (Random Forest), 'knn' (k-Nearest Neighbors), 'svm' (Support Vector Machine).
  - vectorizer (str): Type of vectorizer to test. Choose from 'count' (Count Vecotizer) or 'tfidf' (TF-IDF Vecotizer).
  - ngram (int): Feature representation to test. Choose 1 for unigrams, 2 for bigrams, and so on.
  - search_type (str): Defines grid search or random search style. Choose from 'grid' (Grid Search), 'rand' (Random Search).
  - param_grid (dict): Hyperparameter grid for the specified model and dimensionality reduction method.
  - metric_score (str): Defines metric to score search on. Choose from 'f1' (F-1), 'auc-pr' (area under the precision-recall curve). *We can add more options if needed.

  Returns:
  - selected_model: Trained model with the best hyperparameters.
  - selected_params (dict): Best hyperparameters found during the search.
  - X_train_ (array-like): Vectorized training set features.
  - X_validation_ (array-like): Vectorized validation set features.
  - X_test_ (array-like): Vectorized test set features.
  """
  if vectorizer == 'count':
    vect = CountVectorizer(ngram_range=(ngram, ngram))
    X_train_ = vect.fit_transform(X_train)
    X_validation_ = vect.transform(X_validation)
    X_test_ = vect.transform(X_test)

  elif vectorizer == 'tfidf':
    vect = TfidfVectorizer(ngram_range=(ngram, ngram))
    X_train_ = vect.fit_transform(X_train)
    X_validation_ = vect.transform(X_validation)
    X_test_ = vect.transform(X_test)

  else:
      raise ValueError("Invalid vector type. Use 'count' or 'tfidf'.")

  if model_type == 'log':
      model = LogisticRegression(max_iter=1000, random_state=42)
  elif model_type == 'xgb':
      model = XGBClassifier(random_state=42)
  elif model_type == 'rf':
      model = RandomForestClassifier(random_state=42)
  elif model_type == 'knn':
      model = KNeighborsClassifier()
  elif model_type == 'svm':
      model =  SVC(probability=True)
  else:
      raise ValueError("Invalid model type. Use 'xgb', 'rf', 'svm', 'knn', or 'log'.")

  # Pipeline with dimensionality reduction method and model to test
  #Chose SVD ad reduction method because the data is sparse (PCA and NMF not applicable)
  pipeline = make_pipeline(
    TruncatedSVD(random_state=42),
    model
  )

  # Cross-validation StratifiedKFold for classification (Reduce risk of overfitting)
  cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

  if metric_score == 'f1':
    scorer = make_scorer(f1_score)
  elif metric_score == 'auc-pr':
    scorer = make_scorer(average_precision_score)

  if search_type == 'grid':
    # Use F1-score as the scoring metric in GridSearchCV (This can be changed to any other metric)

    search = GridSearchCV(
        pipeline, param_grid, cv=cv, scoring=scorer, n_jobs=-1
    )
    # Fit the grid search to the data
    search.fit(X_train_, y_train)

  elif search_type == 'random':

    # Use F1-score as the scoring metric in RandomizedSearchCV
    search = RandomizedSearchCV(
      pipeline, param_grid, cv=cv, scoring=scorer, n_iter=10, n_jobs=-1, random_state=42
    )
    # Fit random search to the data
    search.fit(X_train_, y_train)

  else:
    raise ValueError("Invalid search type. Use 'grid' or 'random'.")

  # Get best parameters
  selected_params = search.best_params_
  print(f"Hyperparameters:", selected_params)

  # Train a new model with the best hyperparameters
  selected_model = search.best_estimator_

  # Evaluate the model on the validation set
  y_val_pred = selected_model.predict(X_validation_)
  metrics_val_df, confusion_val_df = model_eval(selected_model, X_validation_, y_validation, y_val_pred)

  return selected_model, selected_params, X_train_, X_validation_, X_test_, metrics_val_df

In [ ]:
def model_rank(model_list, model_str, metric):
    """
    Finds the model with the best score based on a specified metric.

    Parameters:
    - models_list (list): List of dictionaries, each representing a model's details.
    - model_str (list): List of model names corresponding to models_list.
    - metric (str): Metric to rank the models by (e.g., 'Accuracy', 'F1-Score').

    Returns:
    - all_models (pd.DataFrame): DataFrame with metric scores and model names.
    - models_by_metric (pd.DataFrame): DataFrame filtered by the specified metric and sorted in descending order.
    """
    all_models = [model_dict['metrics'].assign(Model=model_name) for model_dict, model_name in zip(model_list, model_str)]

    # Concatenate the DataFrames in the list
    all_models = pd.concat(all_models, ignore_index=True)


    # Sort the DataFrame by the specified metric in descending order
    models_by_metric = all_models[all_models['Metric'] == metric].sort_values(by='Score', ascending=False)

    return all_models, models_by_metric

# Train, Validate, Test Split

In [ ]:
# Annotation only
# Set train-test split variables
X = annotations['response_text']
y = annotations['op_gender_binary']

# Perform train-test split
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# Split the temp set into validation and test sets
X_validation, X_test, y_validation, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)

In [ ]:
# Apply preprocessing to each set (X_train, X_validation, X_test)
X_train_preprocessed = X_train.apply(preprocess_text)
X_validation_preprocessed = X_validation.apply(preprocess_text)
X_test_preprocessed = X_test.apply(preprocess_text)

Check for class imbalance:

In [ ]:
class_distribution = pd.Series(y_train).value_counts()
class_distribution

1    5464
0    5281
Name: op_gender_binary, dtype: int64

## XGBoost

In [ ]:
# Define the parameter grid
param_grid = {
  'truncatedsvd__n_components': [150, 200, 250],              # Number of components to keep after dimensionality reduction using Truncated SVD
  'xgbclassifier__n_estimators': [50, 100, 150],              # Number of boosting rounds (trees) in the XGBoost model
  'xgbclassifier__max_depth': [3, 5, 7],                      # Maximum depth of each tree in the XGBoost model
  'xgbclassifier__learning_rate': [0.01, 0.1, 0.2],           # Step size shrinkage used in boosting (controls the learning rate)
  'xgbclassifier__subsample': [0.8, 1.0],                     # Fraction of samples used for training each tree (subsample ratio)
  'xgbclassifier__colsample_bytree':  [0.8, 1.0],             # Fraction of features used for training each tree (column subsampling ratio)
}

In [ ]:
#Enhanced after random search
param_grid = {
  'truncatedsvd__n_components': [125, 150, 175],          # Enhanced range for the number of components in Truncated SVD
  'xgbclassifier__n_estimators': [130, 150, 170],                      # Enhanced range for the number of boosting rounds in XGBoost
  'xgbclassifier__max_depth': [7, 8, 9],                            # Enhanced range for the maximum depth of each tree in XGBoost
  'xgbclassifier__learning_rate': [0.05, 0.1, 0.15],   # Enhanced range for the learning rate in XGBoost
  'xgbclassifier__subsample': [0.5, 0.6, 0.8],                      # Enhanced range for the subsample ratio in XGBoost
  'xgbclassifier__colsample_bytree':  [.5, .8, 1.0],              # Enhanced range for the column subsampling ratio in XGBoost
}

### XGB Model Method:
*   Vectorization: Count
*   Feature Representation: Unigram



In [ ]:
# Define variables
model = 'xgb'
vectorization = 'count'
ngram = 1
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid,metric_score)

# Save results to dictionary
xgb_count_1 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'xgbclassifier__subsample': 1.0, 'xgbclassifier__n_estimators': 150, 'xgbclassifier__max_depth': 5, 'xgbclassifier__learning_rate': 0.1, 'xgbclassifier__colsample_bytree': 0.8, 'truncatedsvd__n_components': 200}
Metrics:
      Metric     Score
0   Accuracy  0.550825
1  Precision  0.562771
2     Recall  0.551315
3   F1-Score  0.556984
4        AUC  0.550813
5   Log-Loss  0.690280

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                 618                 505
Actual Negative                 529                 650


### XGB Model Method:
*   Vectorization: Count
*   Feature Representation: Bigram

In [ ]:
# Define variables
model = 'xgb'
vectorization = 'count'
ngram = 2
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid,metric_score)

# Save results to dictionary
xgb_count_2 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'xgbclassifier__subsample': 0.8, 'xgbclassifier__n_estimators': 150, 'xgbclassifier__max_depth': 9, 'xgbclassifier__learning_rate': 0.05, 'xgbclassifier__colsample_bytree': 0.5, 'truncatedsvd__n_components': 150}
Metrics:
      Metric     Score
0   Accuracy  0.552997
1  Precision  0.549020
2     Recall  0.712468
3   F1-Score  0.620155
4        AUC  0.549021
5   Log-Loss  0.709090

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                 433                 690
Actual Negative                 339                 840


### XGB Model Method:
*   Vectorization: TF-IDF
*   Feature Representation: Unigram

In [ ]:
# Define variables
model = 'xgb'
vectorization = 'tfidf'
ngram = 1
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test,metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid,metric_score)

# Save results to dictionary
xgb_tfidf_1 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'xgbclassifier__subsample': 0.8, 'xgbclassifier__n_estimators': 150, 'xgbclassifier__max_depth': 9, 'xgbclassifier__learning_rate': 0.02, 'xgbclassifier__colsample_bytree': 0.4, 'truncatedsvd__n_components': 150}
Metrics:
      Metric     Score
0   Accuracy  0.552129
1  Precision  0.558730
2     Recall  0.597116
3   F1-Score  0.577286
4        AUC  0.551007
5   Log-Loss  0.675757

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                 567                 556
Actual Negative                 475                 704


### XGB Final Model:
*   Vectorization: TF-IDF
*   Feature Representation: Unigram

In [ ]:
vect = TfidfVectorizer(ngram_range=(ngram, ngram))
    X_train_ = vect.fit_transform(X_train)
    X_validation_ = vect.transform(X_validation)
    X_test_ = vect.transform(X_test)

test = X_test_final
train = X_train_final
best_hyperparameters = {'xgbclassifier__subsample': 0.8, 'xgbclassifier__n_estimators': 150, 'xgbclassifier__max_depth': 9, 'xgbclassifier__learning_rate': 0.05, 'xgbclassifier__colsample_bytree': 0.5, 'truncatedsvd__n_components': 150}

# Create the final pipeline
final_model_pipeline = make_pipeline(
    TruncatedSVD(n_components=best_hyperparameters['truncatedsvd__n_components'], random_state=42),
    XGBClassifier(
        subsample=best_hyperparameters['xgbclassifier__subsample'],
        n_estimators=best_hyperparameters['xgbclassifier__n_estimators'],
        max_depth=best_hyperparameters['xgbclassifier__max_depth'],
        learning_rate=best_hyperparameters['xgbclassifier__learning_rate'],
        colsample_bytree=best_hyperparameters['xgbclassifier__colsample_bytree'],
        random_state=42
    )
)

# Fit the final model
final_model_pipeline.fit(X_train_final, y_train)

# Evaluate the final model on the test set
y_test_pred = final_model_pipeline.predict(X_test_final)
metrics_test_df, confusion_test_df = model_eval(final_model_pipeline, X_test_final, y_test, y_test_pred)


Metrics:
      Metric     Score
0   Accuracy  0.577073
1  Precision  0.572856
2     Recall  0.598782
3   F1-Score  0.585532
4        AUC  0.577120
5   Log-Loss  0.670886

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                 641                 513
Actual Negative                 461                 688


### XGB Model Method:
*   Vectorization: TF-IDF
*   Feature Representation: Bigram

In [ ]:
# Define variables
model = 'xgb'
vectorization = 'tfidf'
ngram = 2
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid,metric_score)

# Save results to dictionary
xgb_tfidf_2 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'xgbclassifier__subsample': 0.6, 'xgbclassifier__n_estimators': 150, 'xgbclassifier__max_depth': 7, 'xgbclassifier__learning_rate': 0.1, 'xgbclassifier__colsample_bytree': 0.8, 'truncatedsvd__n_components': 150}
Metrics:
      Metric     Score
0   Accuracy  0.553432
1  Precision  0.549443
2     Recall  0.711620
3   F1-Score  0.620103
4        AUC  0.549488
5   Log-Loss  0.726742

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                 435                 688
Actual Negative                 340                 839


## Logistic Regression

In [ ]:
# Define the parameter grid
param_grid = {
    'logisticregression__solver': ['saga'],
    'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__C': [0.001, 0.01, 0.1, 1, 10, 100],  # Adjust the range based on the characteristics of your data
}

### Logistic Regression Model Method:
*   Vectorization: Count
*   Feature Representation: Unigram

In [ ]:
# Define variables
model = 'log'
vectorization = 'count'
ngram = 1
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid, metric_score)

# Save results to dictionary
log_count_1 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'logisticregression__solver': 'saga', 'logisticregression__penalty': 'l1', 'logisticregression__C': 0.1}
Metrics:
      Metric     Score
0   Accuracy  0.525630
1  Precision  0.521610
2     Recall  0.890585
3   F1-Score  0.657895
4        AUC  0.516530
5   Log-Loss  0.692039

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                 160                 963
Actual Negative                 129                1050


### Logistic Regression Model Method:
*   Vectorization: Count
*   Feature Representation: Bigram

In [ ]:
# Define variables
model = 'log'
vectorization = 'count'
ngram = 2
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid, metric_score)

# Save results to dictionary
log_count_2 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'logisticregression__solver': 'saga', 'logisticregression__penalty': 'l1', 'logisticregression__C': 100}
Metrics:
      Metric     Score
0   Accuracy  0.513901
1  Precision  0.513453
2     Recall  0.971162
3   F1-Score  0.671751
4        AUC  0.502500
5   Log-Loss  0.691856

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                  38                1085
Actual Negative                  34                1145


### Logistic Regression Model Method:
*   Vectorization: TF-IDF
*   Feature Representation: Unigram

In [ ]:
# Define variables
model = 'log'
vectorization = 'tfidf'
ngram = 1
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid, metric_score)

# Save results to dictionary
log_tfidf_1 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'logisticregression__solver': 'saga', 'logisticregression__penalty': 'l2', 'logisticregression__C': 100}
Metrics:
      Metric     Score
0   Accuracy  0.523892
1  Precision  0.520039
2     Recall  0.913486
3   F1-Score  0.662769
4        AUC  0.514178
5   Log-Loss  0.692581

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                 129                 994
Actual Negative                 102                1077


### Logistic Regression Model Method:
*   Vectorization: TF-IDF
*   Feature Representation: Bigram

In [ ]:
# Define variables
model = 'log'
vectorization = 'tfidf'
ngram = 2
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid, metric_score)

# Save results to dictionary
log_tfidf_2 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'logisticregression__solver': 'saga', 'logisticregression__penalty': 'l1', 'logisticregression__C': 100}
Metrics:
      Metric     Score
0   Accuracy  0.507819
1  Precision  0.510177
2     Recall  0.977947
3   F1-Score  0.670544
4        AUC  0.496097
5   Log-Loss  0.693502

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                  16                1107
Actual Negative                  26                1153


## Support Vector Machine

In [ ]:
# Define the parameter grid
param_grid = {
    'svc__C': [0.1, 1, 10],              # Regularization parameter
    'svc__kernel': ['linear', 'rbf'],    # Kernel type
    'svc__gamma': ['scale', 'auto'],     # Kernel coefficient
}

### Support Vector Machine Model Method:
*   Vectorization: Count
*   Feature Representation: Unigram

In [ ]:
# Define variables
model = 'svm'
vectorization = 'count'
ngram = 1
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid, metric_score)

# Save results to dictionary
svm_count_1 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'svc__kernel': 'rbf', 'svc__gamma': 'scale', 'svc__C': 10}
Metrics:
      Metric     Score
0   Accuracy  0.526499
1  Precision  0.524930
2     Recall  0.794741
3   F1-Score  0.632254
4        AUC  0.519811
5   Log-Loss  0.692200

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                 275                 848
Actual Negative                 242                 937


### Support Vector Machine Model Method:
*   Vectorization: Count
*   Feature Representation: Bigram

In [ ]:
# Define variables
model = 'svm'
vectorization = 'count'
ngram = 2
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid, metric_score)

# Save results to dictionary
svm_count_2 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'svc__kernel': 'rbf', 'svc__gamma': 'scale', 'svc__C': 10}
Metrics:
      Metric     Score
0   Accuracy  0.516942
1  Precision  0.515276
2     Recall  0.958439
3   F1-Score  0.670225
4        AUC  0.505934
5   Log-Loss  0.692596

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                  60                1063
Actual Negative                  49                1130


### Support Vector Machine Model Method:
*   Vectorization: TF-IDF
*   Feature Representation: Unigram

In [ ]:
# Define variables
model = 'svm'
vectorization = 'tfidf'
ngram = 1
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid, metric_score)

# Save results to dictionary
svm_tfidf_1 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'svc__kernel': 'rbf', 'svc__gamma': 'scale', 'svc__C': 10}
Metrics:
      Metric     Score
0   Accuracy  0.515639
1  Precision  0.518223
2     Recall  0.771841
3   F1-Score  0.620102
4        AUC  0.509251
5   Log-Loss  0.692726

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                 277                 846
Actual Negative                 269                 910


### Support Vector Machine Model Method:
*   Vectorization: TF-IDF
*   Feature Representation: Bigram

In [ ]:
# Define variables
model = 'svm'
vectorization = 'tfidf'
ngram = 2
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid, metric_score)

# Save results to dictionary
svm_tfidf_2 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'svc__kernel': 'rbf', 'svc__gamma': 'scale', 'svc__C': 10}
Metrics:
      Metric     Score
0   Accuracy  0.516073
1  Precision  0.514646
2     Recall  0.968617
3   F1-Score  0.672160
4        AUC  0.504790
5   Log-Loss  0.692515

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                  46                1077
Actual Negative                  37                1142


## K-Nearest Neighbors

In [ ]:
# Define the parameter grid
param_grid = {
    'kneighborsclassifier__n_neighbors': [3, 5, 7],            # Number of neighbors
    'kneighborsclassifier__weights': ['uniform', 'distance'],  # Weight function used in prediction
    'kneighborsclassifier__p': [1, 2],                         # Power parameter for Minkowski metric
}

### K-Nearest Neighbors Model Method:
*   Vectorization: Count
*   Feature Representation: Unigram

In [ ]:
# Define variables
model = 'knn'
vectorization = 'count'
ngram = 1
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid, metric_score)

# Save results to dictionary
knn_count_1 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'kneighborsclassifier__weights': 'distance', 'kneighborsclassifier__p': 2, 'kneighborsclassifier__n_neighbors': 7}
Metrics:
      Metric     Score
0   Accuracy  0.545613
1  Precision  0.557180
2     Recall  0.549618
3   F1-Score  0.553373
4        AUC  0.545513
5   Log-Loss  1.377242

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                 608                 515
Actual Negative                 531                 648


### K-Nearest Neighbors Model Method:
*   Vectorization: Count
*   Feature Representation: Bigram

In [ ]:
# Define variables
model = 'knn'
vectorization = 'count'
ngram = 2
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid, metric_score)

# Save results to dictionary
knn_count_2 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'kneighborsclassifier__weights': 'distance', 'kneighborsclassifier__p': 2, 'kneighborsclassifier__n_neighbors': 7}
Metrics:
      Metric     Score
0   Accuracy  0.544744
1  Precision  0.544407
2     Recall  0.681086
3   F1-Score  0.605124
4        AUC  0.541344
5   Log-Loss  1.114751

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                 451                 672
Actual Negative                 376                 803


### K-Nearest Neighbors Model Method:
*   Vectorization: TF-IDF
*   Feature Representation: Unigram

In [ ]:
# Define variables
model = 'knn'
vectorization = 'tfidf'
ngram = 1
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid, metric_score)

# Save results to dictionary
knn_tfidf_1 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'kneighborsclassifier__weights': 'distance', 'kneighborsclassifier__p': 2, 'kneighborsclassifier__n_neighbors': 7}
Metrics:
      Metric     Score
0   Accuracy  0.519983
1  Precision  0.531040
2     Recall  0.536896
3   F1-Score  0.533952
4        AUC  0.519561
5   Log-Loss  1.330992

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                 564                 559
Actual Negative                 546                 633


### K-Nearest Neighbors Model Method:
*   Vectorization: TF-IDF
*   Feature Representation: Bigram

In [ ]:
# Define variables
model = 'knn'
vectorization = 'tfidf'
ngram = 2
search_type = 'random'
metric_score = 'auc-pr'

# Run model search
model,params,train,validation,test, metrics = model_search(X_train_preprocessed, y_train, X_validation_preprocessed, y_validation, X_test_preprocessed, model, vectorization, ngram, search_type, param_grid, metric_score)

# Save results to dictionary
knn_tfidf_2 = {
    'model': model,
    'params': params,
    'X_train': train,
    'X_validation': validation,
    'X_test': test,
    'metrics': metrics
}

Hyperparameters: {'kneighborsclassifier__weights': 'distance', 'kneighborsclassifier__p': 1, 'kneighborsclassifier__n_neighbors': 3}
Metrics:
      Metric     Score
0   Accuracy  0.551694
1  Precision  0.550034
2     Recall  0.685327
3   F1-Score  0.610272
4        AUC  0.548362
5   Log-Loss  7.951702

Confusion Matrix:
                 Predicted Positive  Predicted Negative
Actual Positive                 462                 661
Actual Negative                 371                 808


# Model Ranking Comparison

In [ ]:
# Model Names (Need string values for dataframe column)
model_list = [xgb_count_1, xgb_tfidf_1, xgb_count_1, xgb_count_2, log_count_1, log_count_2, log_tfidf_1, log_tfidf_2, svm_count_1, svm_count_2, svm_tfidf_1, svm_tfidf_2, knn_count_1, knn_count_2, knn_tfidf_1, knn_tfidf_2]
model_str = ['xgb_count_1', 'xgb_tfidf_1', 'xgb_count_1', 'xgb_count_2', 'log_count_1', 'log_count_2', 'log_tfidf_1', 'log_tfidf_2', 'svm_count_1', 'svm_count_2', 'svm_tfidf_1', 'svm_tfidf_2', 'knn_count_1', 'knn_count_2', 'knn_tfidf_1', 'knn_tfidf_2']

# Specify the metric to rank the models by
all_models, models_by_metric = model_rank(model_list, model_str, 'AUC')
models_by_metric

,Metric,Score,Model
10,AUC,0.571551,xgb_tfidf_1
4,AUC,0.570937,xgb_count_1
16,AUC,0.570937,xgb_count_1
22,AUC,0.549021,xgb_count_2
94,AUC,0.548362,knn_tfidf_2
76,AUC,0.545513,knn_count_1
82,AUC,0.541344,knn_count_2
52,AUC,0.519811,svm_count_1
88,AUC,0.519561,knn_tfidf_1
28,AUC,0.516530,log_count_1


In [ ]:
all_models.to_csv(folder_path+test_num+'all_models.csv', index=False)
models_by_metric.to_csv(folder_path+test_num+'models_by_metric.csv', index=False)